본 문서는 다음의 Kaggle Notebook을 필사했음을 알립니다.  
[NLP Getting Started Tutorial](https://www.kaggle.com/code/philculliton/nlp-getting-started-tutorial)

## NLP Tutorial

- NLP(자연어 처리)는 기계가 텍스트에서 학습하는 데 도움이 되도록 설계된 다양한 기술의 약어입니다.  
<br>
    - 자연어 처리는 챗봇에서 검색 엔진에 이르기까지 모든 것을 지원하며 감정 분석 및 기계 번역과 같은 다양한 작업에 사용됩니다.  
<br>
- 이 튜토리얼에서는 이 대회의 데이터셋을 살펴보고, 이를 처리하는 간단한 기술을 사용하고, 기계 학습 모델을 구축하고, 점수에 대한 예측을 제출할 것입니다.

In [1]:
import numpy as np 
import pandas as pd 
from sklearn import feature_extraction, linear_model, model_selection, preprocessing

In [4]:
train_df = pd.read_csv("D:/DataSet/Natural Language Processing with Disaster Tweets/train.csv")
test_df = pd.read_csv("D:/DataSet/Natural Language Processing with Disaster Tweets/test.csv")

In [5]:
train_df[train_df["target"] == 0]["text"].values[1]

'I love fruits'

In [6]:
train_df[train_df["target"] == 1]["text"].values[1]

'Forest fire near La Ronge Sask. Canada'

### 벡터 구축

- 이 노트북에서 구축할 모델의 이론은 매우 간단합니다. 각 트윗에 포함된 단어는 실제 재해에 대한 것인지 여부를 나타내는 좋은 지표입니다(완전히 정확하지는 않지만 시작하기에 좋음).  
<br>
- scikit-learn의 CountVectorizer를 사용하여 각 트윗의 단어 수를 계산하고 이를 기계 학습 모델이 처리할 수 있는 데이터로 변환합니다.  
<br>
- 참고: 이 컨텍스트에서 벡터는 기계 학습 모델이 작업할 수 있는 숫자 집합입니다. 잠시 후에 하나를 살펴보겠습니다

In [7]:
count_vectorizer = feature_extraction.text.CountVectorizer()

# 데이터의 처음 5개 트윗에 대한 개수
example_train_vectors = count_vectorizer.fit_transform(train_df["text"][0:5])

In [8]:
# 이러한 벡터가 "희소"하기 때문에 여기에서 .todense()를 사용합니다(공간 절약을 위해 0이 아닌 요소만 유지됨)
print(example_train_vectors[0].todense().shape)
print(example_train_vectors[0].todense())

(1, 54)
[[0 0 0 1 1 1 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0
  0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 1 0]]


위의 내용은 다음과 같이 알려줍니다.

처음 5개의 트윗에는 54개의 고유한 단어(또는 "토큰")가 있습니다.
첫 번째 트윗에는 고유한 토큰 중 일부만 포함되어 있습니다. 위의 0이 아닌 모든 수는 첫 번째 트윗에 존재하는 토큰입니다.
이제 모든 트윗에 대한 벡터를 생성해 보겠습니다.

In [12]:
train_vectors = count_vectorizer.fit_transform(train_df["text"])
test_vectors = count_vectorizer.transform(test_df["text"])

여기서는 .fit_transform()을 사용하지 않습니다. .transform()만 사용하면 기차 벡터의 토큰이 테스트 벡터에 매핑되는 유일한 토큰인지 확인합니다. 즉, 기차와 테스트 벡터가 동일한 토큰 세트를 사용합니다.

### 선정 모델

위에서 언급했듯이 각 트윗에 포함된 단어는 실제 재해에 대한 것인지 여부를 나타내는 좋은 지표라고 생각합니다. 트윗에 특정 단어(또는 단어 집합)가 있으면 해당 트윗이 진짜인지 여부와 직접 연결될 수 있습니다.

여기서 우리가 가정하는 것은 선형 연결입니다. 선형 모델을 만들고 살펴봅시다!

벡터가 크기 때문에 다른 단어를 완전히 덜어내지 않고 모델의 가중치를 0으로 넣고 싶습니다.  
<br>

**Ridge regression** 는 이를 수행하는 좋은 방법입니다.

In [10]:
clf = linear_model.RidgeClassifier()

모델을 테스트하고 훈련 데이터에서 얼마나 잘 작동하는지 봅시다. 이를 위해 우리는 교차 검증을 사용할 것입니다.  
여기에서 알려진 데이터의 일부에 대해 학습한 다음 나머지로 검증합니다.  
이것을 여러 번(다른 부분으로) 수행하면 특정 모델이나 방법이 어떻게 수행되는지에 대한 좋은 아이디어를 얻을 수 있습니다.  

이 대회의 메트릭은 F1이므로 여기에서 사용하겠습니다.

In [13]:
scores = model_selection.cross_val_score(clf, train_vectors, train_df["target"], cv=3, scoring="f1")
scores

array([0.59421842, 0.56498283, 0.64082434])

위의 점수는 끔찍하지 않습니다! 우리의 가정은 리더보드에서 대략 0.65점을 받을 것 같습니다. 잠재적으로 이를 개선할 수 있는 방법이 많이 있습니다(TFIDF, LSA, LSTM/RNN, 목록이 깁니다!) - 그 중 아무거나 시도해 보세요!

그동안 훈련 세트에 대한 예측을 수행하고 경쟁에 대한 제출물을 작성해 보겠습니다.

In [15]:
clf.fit(train_vectors, train_df["target"])

RidgeClassifier()

In [16]:
sample_submission = pd.read_csv("D:/DataSet/Natural Language Processing with Disaster Tweets/sample_submission.csv")

In [17]:
sample_submission["target"] = clf.predict(test_vectors)

In [18]:
sample_submission.head()

,id,target
0,0,0
1,2,1
2,3,1
3,9,0
4,11,1


In [20]:
sample_submission.to_csv("D:/DataSet/Natural Language Processing with Disaster Tweets/submission.csv", index=False)

이제 뷰어에서 위 파일을 공모전에 제출하시면 됩니다! 행운을 빕니다!